In [ ]:
#this integration is being created to run the kaggle diamond dataset machine learning example (dataset here in kaggle):
#https://www.kaggle.com/datasets/shivam2503/diamonds?resource=download

# Snowpark for Python
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType

import numpy as np
# Override np.float_ with np.float64
np.float_ = np.float64

# Snowpark ML
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.metrics.correlation import correlation

# Data science libs
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib

#warning suppression
import warnings; warnings.simplefilter('ignore')

# Get active session (current snowflake session)
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# add a query tag to the session 
session.query_tag = {"origin":"sf_sit-is","name":"e2e_ml_snowparkpython", "version":{"major":1,"minor":0}}